The purpose of this project is to generate evidence using Evidence Model.

---

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [11]:
content = []
with open('/content/gdrive/MyDrive/EvidenceQuery/debatepedia/train_content', "r") as d:
  for doc in d:
    content.append(doc[4:-7])

query = []
with open('/content/gdrive/MyDrive/EvidenceQuery/debatepedia/train_query', "r") as d:
  for doc in d:
    query.append(doc[4:-7])

summary = []
with open('/content/gdrive/MyDrive/EvidenceQuery/debatepedia/train_summary', "r") as d:
  for doc in d:
    summary.append(doc[4:-7])


In [4]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [5]:
!pip install transformers
!pip install sentencepiece==0.1.91

     |████████████████████████████████| 3.4 MB 8.4 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 3.3 MB 57.2 MB/s 
     |████████████████████████████████| 895 kB 67.8 MB/s 
     |████████████████████████████████| 596 kB 70.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.1 MB 7.0 MB/s 


In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained('/content/gdrive/MyDrive/EvidenceQuery/model').to(device)
tokenizer = T5Tokenizer.from_pretrained('t5-base')

def generateEvidence (text_document):
  input_ids = tokenizer.encode(text_document, padding=True, truncation = True, return_tensors='pt').to(device)
  output = model.generate(input_ids, max_length=12, min_length=12, no_repeat_ngram_size= 4)
  evidence = tokenizer.decode(output[0], skip_special_tokens=True)
  return evidence

#print(summary[200])
#print(generateEvidence(content[200]))

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

In [12]:
from tqdm import tqdm
evidence = []

for i in tqdm(range(len(content))):
   evidence.append(generateEvidence(content[i]))

100%|██████████| 12000/12000 [31:31<00:00,  6.34it/s]


In [13]:
import json

debatepedia_data = []

for i in range(len(content)):
  debatepedia_data.append({'src':content[i],'qry':query[i],'tgt':summary[i],'evd':evidence[i]})

with open('/content/gdrive/MyDrive/EvidenceQuery/debatepediaEvidence_train.json', 'w') as fp:
  json.dump(debatepedia_data, fp)